In [20]:
from typing import Optional
import gmpy2
import random
import math
from itertools import chain, combinations

In [21]:
PRIMES = primes = (2,3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97,
                   101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193,
                   197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307,
                   311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421,
                   431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547,
                   557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 
                   661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 
                   809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929,
                   937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 
                   1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171,
                   1181, 1187, 1193, 1201, 1213, 1217, 1223, 1229, 1231, 1237, 1249, 1259, 1277, 1279, 1283, 1289, 1291,
                   1297, 1301, 1303, 1307, 1319, 1321, 1327, 1361, 1367, 1373, 1381, 1399, 1409, 1423, 1427, 1429, 1433,
                   1439, 1447, 1451, 1453, 1459, 1471, 1481, 1483, 1487, 1489, 1493, 1499, 1511, 1523, 1531, 1543, 1549,
                   1553, 1559, 1567, 1571, 1579, 1583, 1597, 1601, 1607, 1609, 1613, 1619, 1621, 1627, 1637, 1657, 1663,
                   1667, 1669, 1693, 1697, 1699, 1709, 1721, 1723, 1733, 1741, 1747, 1753, 1759, 1777, 1783, 1787, 1789,
                   1801, 1811, 1823, 1831, 1847, 1861, 1867, 1871, 1873, 1877, 1879, 1889, 1901, 1907, 1913, 1931, 1933,
                   1949, 1951, 1973, 1979, 1987, 1993, 1997, 1999, 2003, 2011, 2017, 2027, 2029, 2039, 2053, 2063, 2069,
                   2081, 2083, 2087, 2089, 2099, 2111, 2113, 2129, 2131, 2137, 2141, 2143, 2153, 2161, 2179, 2203, 2207,
                   2213, 2221, 2237, 2239, 2243, 2251, 2267, 2269, 2273, 2281, 2287, 2293, 2297, 2309, 2311, 2333, 2339,
                   2341, 2347, 2351, 2357, 2371, 2377, 2381, 2383, 2389, 2393, 2399, 2411, 2417, 2423, 2437, 2441, 2447,
                   2459, 2467, 2473, 2477, 2503, 2521, 2531, 2539, 2543, 2549, 2551, 2557, 2579, 2591, 2593, 2609, 2617,
                   2621, 2633, 2647, 2657, 2659, 2663, 2671, 2677, 2683, 2687, 2689, 2693, 2699, 2707, 2711, 2713, 2719,
                   2729, 2731, 2741, 2749, 2753, 2767, 2777, 2789, 2791, 2797, 2801, 2803, 2819, 2833, 2837, 2843, 2851,
                   2857, 2861, 2879, 2887, 2897, 2903, 2909, 2917, 2927, 2939, 2953, 2957, 2963, 2969, 2971, 2999)

In [22]:
def product_of_primes(nprimes):  
    product_of_primes = 1
    if (nprimes>len(PRIMES)): nprimes=len(PRIMES)
    for i in range(0,nprimes):
        product_of_primes *= primes[i]
    return product_of_primes

def Pollardpm1(n: int, m: Optional[int] = None):
    
        # Using a = 2^(n-1) instead of a = 2 to also handle cases in which
        # p-1 = b*m or q-1 = c*m, where either b or c is smooth
        a = pow(2, n - 1, n)
        p = gmpy2.gcd(pow(a, m, n) - 1, n)
        # gcd is actually p
        if 1 < p < n:
            return True, [p, n // p]
        # when both  p-1 and q-1 are smooth enough, we should try with divisors of m
        if p == n:
            print("Trying to factorize with divisors of m...\n")
            res, factors = factorizeWithDivisorOfM(a, n)
            return res, factors
        # The factorization fails when p-1 is not 2999-powersmooth
        # m with more and larger prime factors can be chosen, but will take much longer
        return False, []

def factorizeWithDivisorOfM(a, n):
    # trying using product of subsets of PRIMES as new estimate of k*(p-1)
    for result in powerset(PRIMES):
        m = math.prod(result)
        p = gmpy2.gcd(pow(a, m, n) - 1, n)
        if 1 < p < n:
            return True, [p, n // p]
        else:
            continue
    return False, []
    
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [23]:
nprimebits=32
nprimes = 429 # len(PRIMES) - 1 i.e. p should be factorizable into prime integers < 2999
print(f"Number of bits in prime={nprimebits} and {primes[nprimes]}-powersmooth\n")

Number of bits in prime=32 and 2999-powersmooth



In [24]:
# Poor factor values that can be broken by Pollard n-1 attack
# p=3762285479, q=2263720993 ; p=1233011711, q=2692517827 ; p=3811812467, q=4260289277 ; p=107754971, q=1277844937
# p=11257, q=25237 ; p=1504433947, q=4000118389 ; p=2389807349, q=3053820643 ; p=715069879, q=2525059253

p = gmpy2.next_prime(random.getrandbits(nprimebits))
q = gmpy2.next_prime(random.getrandbits(nprimebits))
n = p*q

# m -> Using product of PRIMES as a pre-calculated guess for K*(p-1)
m = product_of_primes(nprimes)
res, factors = Pollardpm1(n, m)
print(f"Result: ", res)
if (len(factors)>1):
    print(f"Factors found: {int(factors[0])}, {int(factors[1])}")
else:
    print("No factors discovered")
    
print(f"\nThe originally generator factors were: p={p}, q={q}")

Result:  True
Factors found: 861205577, 129793847

The originally generator factors were: p=129793847, q=861205577
